
<a href="https://colab.research.google.com/github/google-research/albert/blob/master/albert_glue_fine_tuning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Copyright 2020 The ALBERT Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# ALBERT End to End (Fine-tuning + Predicting) with Cloud TPU

## Overview

ALBERT is "A Lite" version of BERT, a popular unsupervised language representation learning algorithm. ALBERT uses parameter-reduction techniques that allow for large-scale configurations, overcome previous memory limitations, and achieve better behavior with respect to model degradation.

For a technical description of the algorithm, see our paper:

https://arxiv.org/abs/1909.11942

Zhenzhong Lan, Mingda Chen, Sebastian Goodman, Kevin Gimpel, Piyush Sharma, Radu Soricut

This Colab demonstates using a free Colab Cloud TPU to fine-tune GLUE tasks built on top of pretrained ALBERT models and 
run predictions on tuned model. The colab demonsrates loading pretrained ALBERT models from both [TF Hub](https://www.tensorflow.org/hub) and checkpoints.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All** (Watch out: the "Colab-only auth for this notebook and the TPU" cell requires user input). You can also run the cells manually with Shift-ENTER.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [ ]:
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf
%load_ext tensorboard

from google.colab import auth
auth.authenticate_user()

if "COLAB_TPU_ADDR" in os.environ :
  # assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
  TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
  TPU_TOPOLOGY = "2x2"
  print("TPU address is", TPU_ADDRESS)

  with tf.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    pprint.pprint(session.list_devices())

    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)


### Prepare and import ALBERT modules
​
With your environment configured, you can now prepare and import the ALBERT modules. The following step clones the source code from GitHub.

In [ ]:
#TODO(lanzhzh): Add pip support
import sys
!rm -rf sample_data
!test -d albert || git clone https://github.com/google-research/albert albert
if not 'albert' in sys.path:
  sys.path += ['albert']

!test -d albert-utils || git clone https://github.com/laouer/albert-utils.git albert-utils

!pip install sentencepiece progressbar



### Prepare for training
This next section of code performs the following tasks:
*  Specify GS bucket, create output directory for model checkpoints and eval results.
*  Make sure that the Input files are located on the GS bucket
*  Specify ALBERT pretrained model

In [ ]:
BUCKET = "" #@param { type: "string" }
# Available pretrained model checkpoints:
#   base, large, xlarge, xxlarge
ALBERT_MODEL = "base" #@param ["base", "large","xlarge","xxlarge"]

BASE_DIR = "gs://" + BUCKET
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BUCKET.")

MODELS_DIR = os.path.join(BASE_DIR, "models")
OUTPUT_DIR = 'gs://{}/albert-tfhub/models/WIKIFR/{}'.format(BUCKET, ALBERT_MODEL)
FEATURE_DIR = 'gs://{}/albert-tfhub/models/WIKIFR/wikifr_tf_records/'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

ALBERT_MODEL_HUB_NAME="albert_"+ ALBERT_MODEL + "_3"
ALBERT_MODEL_HUB = 'https://tfhub.dev/google/albert_' + ALBERT_MODEL + '/3'

!wget -q --content-disposition  $ALBERT_MODEL_HUB"?tf-hub-format=compressed" && \
      mkdir -p $ALBERT_MODEL_HUB_NAME && \
      tar -xf $ALBERT_MODEL_HUB_NAME".tar.gz" -C $ALBERT_MODEL_HUB_NAME && \
      rm -rf $ALBERT_MODEL_HUB_NAME".tar.gz"

ALBERT_CONFIG_FILE = ALBERT_MODEL_HUB_NAME + "/assets/albert_config.json"
INIT_CHECKPOINT_FILE=ALBERT_MODEL_HUB_NAME + "/variables/"
INPUT_FILES_PATTERN=FEATURE_DIR + "*/*.tf_record"

print('***** Input files pattern: {} *****'.format(INPUT_FILES_PATTERN))

%tensorboard --logdir $OUTPUT_DIR


In [ ]:
# max_seq_length and max_predictions_per_seq must be the same as use for create_pretraining_data

max_seq_length=128
masked_lm_prob=0.2
max_predictions_per_seq=int(masked_lm_prob*max_seq_length)

!python -m albert.run_pretraining \
  --albert_config_file=$ALBERT_CONFIG_FILE \
  --input_file=$INPUT_FILES_PATTERN \
  --output_dir=$OUTPUT_DIR\
  --export_dir=$OUTPUT_DIR"/export/"\
  --max_seq_length=$max_seq_length \
  --max_predictions_per_seq=$max_predictions_per_seq \
  --train_batch_size=1024 \
  --num_train_steps=200000 \
  --do_eval=True \
  --tpu_name=$TPU_ADDRESS \
  --use_tpu=True 